# Module Imports

In [27]:
import os

import pandas as pd
import numpy as np

# Loading the data

##### Using OS module to determine filepaths in OS independent manner as backslashes used on Windows and forward slashes used on Unix

In [28]:
working_directory = os.path.abspath('.')

In [29]:
training_data_location = os.path.join(working_directory, 'Data', 'train.csv')
testing_data_location = os.path.join(working_directory, 'Data', 'test.csv')

In [30]:
titanic_training_data = pd.read_csv(training_data_location)
titanic_testing_data = pd.read_csv(testing_data_location)

In [31]:
len(titanic_testing_data)

418

# Initial data exploration

In [32]:
titanic_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Training data consist of the following columns:

    1) Passenger ID: Assuming this is unique
    2) Survived: This is a true(1)/false(0) of whether or not the passenger has survived
    3) Pclass: class of ticket, this is either lower (3), middle (2) or upper (1)
    4) Name: Passenger name as a string. It may be worth splitting passenger name into surnames to see if we can group by families
    5) Sex: categorical of either male or female
    6) SibSp: This is the number of siblings and spouses on-board the titanic
    7) Parch: This is the number of parents and children on-board the titanic
    8) Ticket number: I assume this is unique --> I wonder if there is any processing that can be done to extract more information.
    9) Fare: price paid for ticket
    10) Cabin: Cabin number, maybe this is something to do with location upon the titanic..may require some processing. Or maybe we can make a new column which represents whether or not a person has a Cabin.
    11) Embarked: Location at which person embarked the titanic. S is Southampton, Q is queenstown and C is Cherbourg


## Data Preparation

Based off previous analysis the Cabin column needs to be processed to obtain information about whether or not a person had a cabin and also the location of their cabin.

In [33]:
titanic_training_data['Cabin'] = titanic_training_data.Cabin.str.get(0).fillna('NoCabin')
titanic_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NoCabin,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NoCabin,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NoCabin,S


In [34]:
titanic_testing_data['Cabin'] = titanic_testing_data.Cabin.str.get(0).fillna('NoCabin')
titanic_testing_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NoCabin,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NoCabin,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NoCabin,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NoCabin,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NoCabin,S


In [35]:
titanic_training_data['Family_Name'] = titanic_training_data.Name.str.split(',').str.get(0)
titanic_testing_data['Family_Name'] = titanic_testing_data.Name.str.split(',').str.get(0)

Worked out family names incase I want to include that in model.

## First Model

As I don't know what type of estimator I should be using I used the SciKit-Learn cheat sheet to choose LinearSVC (https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html). The reason for this is that we have available to us more then 50 labelled training set rows and we wish to classify them into two categories which are whether or not a person survived.

SVC stands for support vector classification. It looks to find a plane betweeen the extremes of different categories.

#### Importing SVM library

In [56]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.impute  import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#### Data preprocessing

#### First remove all NaNs

In [37]:
#titanic_training_data = titanic_training_data.dropna()
#titanic_testing_data = titanic_testing_data.dropna()

#### Remove Cabin T as looks like outlier.

In [38]:
titanic_training_data = titanic_training_data[titanic_training_data.Cabin!='T']

First want to decide which features to use. Based off previous analysis I would like to use:

    1) Sex
    2) Ticket class
    3) Siblings and spouses
    4) Parents and children
    5) Age
    6) Cabin
    7) Fare cost

In [39]:
#feature_columns = ['Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Age', 'Embarked']
#categorical_columns = ['Cabin', 'Sex', 'Embarked']
#X_data = titanic_training_data[feature_columns]
#X_data_kaggle = titanic_testing_data[feature_columns]
#Y_data = titanic_training_data['Survived']

In [113]:
feature_columns = ['Sex', 'Cabin']
categorical_columns = ['Cabin', 'Sex']
X_data = titanic_training_data[feature_columns]
X_data_kaggle = titanic_testing_data[feature_columns]
Y_data = titanic_training_data['Survived']

Encode categorical data

In [114]:
X_data = pd.get_dummies(X_data, drop_first=False)
X_data_kaggle = pd.get_dummies(X_data_kaggle, drop_first=False)
X_data.head(1)

,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_NoCabin
0,0,1,0,0,0,0,0,0,0,1


The X data is encoded, however, need to drop one column from each category.

In [116]:
columns_to_drop = ['Sex_male', 'Cabin_A']
X_data = X_data.drop(columns_to_drop, axis=1)
X_data_kaggle = X_data_kaggle.drop(columns_to_drop, axis=1)
X_data.head(1)

,Sex_female,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_NoCabin
0,0,0,0,0,0,0,0,1


In [117]:
X_data_kaggle.head(1)

,Sex_female,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_NoCabin
0,0,0,0,0,0,0,0,1


#### Impute the data to remove NaNs

In [118]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [119]:
imputer.fit(X_data)

SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)

In [120]:
X_data = imputer.transform(X_data)
X_data_kaggle = imputer.transform(X_data_kaggle)

In [121]:
scaler = StandardScaler()

In [122]:
processed_X_data = scaler.fit_transform(X_data)

In [123]:
processed_X_data_kaggle = scaler.transform(X_data_kaggle)

#### Fitting a Linear SVM model

##### Splitting the data into training and testing set

In [124]:
X_train, X_test, Y_train, Y_test = train_test_split(processed_X_data, Y_data, test_size = 0.2, random_state = 99)

In [125]:
classifier = LinearSVC(max_iter = 10000)

In [126]:
classifier.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=10000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [127]:
y_pred = classifier.predict(X_test)

In [128]:
from sklearn.metrics import confusion_matrix

In [129]:
confusion_matrix(y_pred, Y_test)

array([[74, 25],
       [33, 46]], dtype=int64)

Predict Kaggle Dataset

In [130]:
y_pred_kaggle = classifier.predict(X_data_kaggle)

In [131]:
kaggle_final_df = pd.DataFrame({'PassengerID':titanic_testing_data.PassengerId,'Survived':y_pred_kaggle})
kaggle_final_df.head()

,PassengerID,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Export dataframe to csv

In [132]:
kaggle_final_df.to_csv('linar_SVM_prediction.csv', index=False)

### Decision Tree Classifier

In [96]:
from sklearn.tree import DecisionTreeClassifier

In [103]:
tree_classifier = DecisionTreeClassifier(random_state=0)

In [104]:
tree_classifier.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [108]:
y_pred = classifier.predict(X_test)

In [109]:
confusion_matrix(y_pred, Y_test)

array([[80, 24],
       [27, 47]], dtype=int64)

In [110]:
y_pred_kaggle = classifier.predict(X_data_kaggle)

In [111]:
kaggle_final_df = pd.DataFrame({'PassengerID':titanic_testing_data.PassengerId,'Survived':y_pred_kaggle})
kaggle_final_df.head()

,PassengerID,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [112]:
kaggle_final_df.to_csv('decision_tree_SVM_prediction.csv', index=False)